# Spectrograms NN 

First import the necessary packages 

In [2]:
import numpy as np
import torch
import sys
import time

import torch.optim as optim
import torch.nn as nn
from torch.utils import data
import torch.nn.functional as F
from torchvision import transforms


cuda = torch.cuda.is_available()
cuda


# data = np.load('dev.npy', allow_pickle = True)
# datacat = np.concatenate(data, axis = 0)
# print(datacat[slice(100,102)])

True

### Dataloader

Will be getting two dataset, a train and labels.

In [3]:
class MyDataset(data.Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
        print(np.load(self.X, allow_pickle = True).shape)
        self.Xcat = np.concatenate(np.load(self.X, allow_pickle = True), axis = 0)
        self.Ycat = np.concatenate(np.load(self.Y, allow_pickle = True), axis = 0)

        self.k = 0
        if self.k > 0:
          zeros = np.zeros((self.k, 40)).astype(float)
          self.Xcat = np.append(zeros, self.Xcat, axis = 0)  
          self.Xcat = np.append(self.Xcat, zeros, axis = 0)       

        self.x = self.Xcat
        self.y = np.reshape(self.Ycat, (len(self.Ycat), 1))
        print(self.x.shape, self.y.shape)

    def __len__(self):
        return len(self.Y)

    def __getitem__(self,index):
        # turn Y to long to pass into cross entropy?

        X = self.x[slice(index-self.k, index+self.k+1)].reshape(-1).astype(float)
        Y = self.y[index].astype(int)
        return X, Y

    # dataset loader, turn data into 2D array, then slice.  

num_workers = 4 if cuda else 0 
    
# Training
train_dataset = MyDataset('dev.npy', 'dev_labels.npy')

train_params = dict(shuffle=True, batch_size=64, num_workers=num_workers, pin_memory=False) if cuda\
                    else dict(shuffle=True, batch_size=64)

train_load = data.DataLoader(train_dataset, **train_params)

# # Testing
# test_dataset = MyDataset('dev.npy', 'dev_labels.npy')

# test_params = dict(shuffle=False, batch_size=256, num_workers=num_workers, pin_memory=True) if cuda\
#                     else dict(shuffle=False, batch_size=1)

# test_load = data.DataLoader(test_dataset, **test_params)

(1100,)
(675836, 40) (675836, 1)


In [4]:
for (x, y) in train_load:
    print(type(x[0]),x[0])
    break

<class 'torch.Tensor'> tensor([-276.2644, -165.0078, -204.2537, -166.4714, -206.1901, -235.2212,
        -189.5758, -218.4456, -168.3412, -212.7154, -175.5260, -258.9223,
        -153.3019, -195.4387, -239.7365, -232.7189, -182.9045, -263.9122,
        -217.3967, -275.5172, -168.0723, -238.3570, -220.0867, -232.6366,
        -247.5462, -226.2658, -188.4547, -262.8099, -202.5450, -200.9244,
        -261.6183, -122.4430, -209.3424, -190.1110, -199.0182, -215.4001,
        -236.8663, -221.0992, -179.4614, -217.6890], dtype=torch.float64)


### Define a NN Class

In [5]:
class simpleMLP(nn.Module):
    def __init__(self, linear_layers, num_batch_norm):
        super(simpleMLP, self).__init__()

        layers = []
        self.linear_layers = linear_layers
        self.num_batch_norm = num_batch_norm
        self.bn = num_batch_norm > 0

        # create the layers
        for i in range(len(linear_layers) - 2):
            layers.append(nn.Linear(linear_layers[i], linear_layers[i+1]))
            if self.bn and i < num_batch_norm:
                layers.append(nn.BatchNorm1d(linear_layers[i+1]))
            layers.append(nn.Sigmoid())
        layers.append(nn.Linear(linear_layers[-2], linear_layers[-1]))

        # combine final net and assign
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

### Create the model and define our loss criterion

May want to add optimal learning rate schedule


In [6]:
model = simpleMLP([40, 256, 138], 1)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())
device = torch.device("cuda" if cuda else "cpu")
model.to(device)
print(model, criterion)

simpleMLP(
  (net): Sequential(
    (0): Linear(in_features=40, out_features=256, bias=True)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Sigmoid()
    (3): Linear(in_features=256, out_features=138, bias=True)
  )
) CrossEntropyLoss()


### Our training function

Basically we do a foward pass, then compute the loss, backward pass, take a step and reset.

In [7]:
def train_epoch(model, train_load, criterion, optimizer):
    # indicate that we are training 
    model.train()
    running_loss = 0.0
    
    # start timer and start iterating
    start_train = time.time()
    for batch_idx, (data, target) in enumerate(train_load):   
        data = data.to(device)
        target = target.to(device) # all data & model on same device
        
        # forward, then backward, then step
        outputs = model(data.float())
        print(outputs)
        loss = criterion(torch.LongTensor(outputs), target)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()   # .backward() accumulates gradients

        # accumulate loss
        running_loss += loss.item()
    
    # end timer and take average loss
    end_train = time.time()
    running_loss /= len(train_load)
    
    return running_loss, end_train, start_train

### Writing our test function

In [8]:
def test_model(model, test_load, criterion, testout):
    with torch.no_grad():
        # indicate that we are evaluating model
        model.eval()
        
        # initialize errors to 0
        running_loss = 0.0
        total_predictions = 0.0
        correct_predictions = 0.0    
        
        # start iterating
        for batch_idx, (data, target) in enumerate(test_load):   
            data = data.to(device)
            target = target.to(device)

            # run forward pass and then compute loss
            outputs = model(data)
            loss = criterion(outputs, target).detach()
            
            # get predictions 
            _, predicted = torch.max(outputs.data, 0)
            
            # write predictions and calculate correct predictions / loss
            testout.write(str(predicted), '\n')
            total_predictions += target.size(0)
            correct_predictions += (predicted == target).sum().item()
            running_loss += loss.item()
        
        # calculate average loss and accuract
        running_loss /= len(test_load)
        acc = (correct_predictions/total_predictions)*100.0

        return running_loss, acc

### Initialize and run our model

In [ ]:
n_epochs = 1

Train_loss = []
Test_loss = []
Test_acc = []

testout = open("submission.csv", "w")

for i in range(n_epochs):
    
    # Training and outputting loss
    train_loss, end_train, start_train = train_epoch(model, train_load, criterion, optimizer)
    print('Training Loss: ', train_loss, 'Time: ',end_train - start_train, 's')
    
     # Test and ouputting
     test_loss, test_acc = test_model(model, test_load, criterion, testout)
     print('Testing Loss: ', test_loss)
     print('Testing Accuracy: ', test_acc, '%')    
    
     Train_loss.append(train_loss)
     Test_loss.append(test_loss)
     Test_acc.append(test_acc)

     print('='*20)
